In [ ]:
import numpy as np

## 撰寫一個程式去實作 FIFO LRU OPT替換演算法 

In [ ]:
def random_demo(demo, rate, times):
    ### 從demo中以機率(rate)取幾次(times)
    indexes = np.array([], dtype=np.int)

    for i in range(times):
        # 隨機取得demo索引值
        indexes = np.hstack((indexes, np.where(np.random.rand(*demo.shape) < rate)[0]))

    return demo[indexes]

## 分頁號碼從0～9 。

In [ ]:
# 分頁編號 0~9
pages_demo = np.arange(10)
print('pages demo  =', pages_demo)

pages demo  = [0 1 2 3 4 5 6 7 8 9]


## 每一種演算法都使用此亂數的頁參考串。

In [ ]:
# 亂數抽取
pages = random_demo(pages_demo, 0.2, 10)
# pages = np.array([1,0,1,0,2,4,1,0,0,8,7,5,4,3,2,3,4])
print('pages       =', pages)

pages       = [7 8 1 0 1 4 7 4 3 5 4 6 2 3 4 5 6 1 2 4]


In [ ]:
def simulation(pages, cache_size, arg_fn):
    cache      = []
    hit_cache  = []
    fail_cache = []
    exec_times = len(pages)

    print('{}\t| {} <= {}'.format('是否擊中', '分頁編號', '輸入'))
    print('{}\t| {} <= {}'.format('hit/fail', [i+1 for i in range(cache_size)], 'page'))
    print('-'*100)

    for i in range(exec_times):
        page = pages[i]
        chker = (cache == page).any()
        if chker:
            print('{}\t\t| {} <= {}'.format('hit', cache, page))
            hit_cache.append(page)
        else:
            print('{}\t\t| {} <= {}'.format('fail', cache, page))
            if len(cache) < cache_size:
                cache.append(page)
            else:
                cache = arg_fn(i, pages, cache)
            fail_cache.append(page)

    print('-'*100)
    print('hit  cache  = ', hit_cache)
    print('hit  times  = ', len(hit_cache))
    print('hit  rate   = {:.2f}%'.format(len(hit_cache)  * 100 / exec_times))
    print('fail cache  = ', fail_cache)
    print('fail times  = ', len(fail_cache))
    print('fail rate   = {:.2f}%'.format(len(fail_cache) * 100 / exec_times))

## FIFO 算法

In [ ]:
def fifo(index, pages, cache):
    cache.pop(0)
    cache.append(pages[index])
    return cache

## LRU算法

In [ ]:
def lru(index, pages, cache):
    ### 尋找與cache中重複的pages
    calc_same = np.abs(np.array(cache) - pages[:index, None]) == 0
    if calc_same.shape[0] > 0:
        tmp = pages[:index] # 從現在位置向前看
        bk_sort = []
        # 以現在位置在tmp中尋找cahce中過去時間最久的
        for page in tmp[::-1]:
            if not page in bk_sort and page in cache:
                bk_sort.append(page)
        if len(bk_sort) > 0:
            cache.remove(bk_sort[-1])
            cache.append(pages[index])
        return cache
    return fifo(index, pages, cache)

## Opt算法

In [ ]:
def opt(index, pages, cache):
    ### 尋找與cache中重複的pages
    calc_same = np.abs(np.array(cache) - pages[(index + 1):, None]) == 0
    if calc_same.shape[0] != 0:
        tmp = pages[(index + 1):] # 從現在位置向後看
        bk_sort = []
        # 以現在位置在tmp中尋找不會用到的cache
        for page in cache:
            if not page in bk_sort and not page in tmp:
                bk_sort.append(page)
        if len(bk_sort) > 0:
            cache.remove(bk_sort[0])
            cache.append(pages[index])
            return cache
    return fifo(index, pages, cache)

## 執行替換演算法讓分頁欄的編號從1～7。

In [ ]:
print('FIFO')
simulation(pages, 7, arg_fn = fifo)

FIFO
是否擊中	| 分頁編號 <= 輸入
hit/fail	| [1, 2, 3, 4, 5, 6, 7] <= page
----------------------------------------------------------------------------------------------------
fail		| [] <= 7
fail		| [7] <= 8
fail		| [7, 8] <= 1
fail		| [7, 8, 1] <= 0
hit		| [7, 8, 1, 0] <= 1
fail		| [7, 8, 1, 0] <= 4
hit		| [7, 8, 1, 0, 4] <= 7
hit		| [7, 8, 1, 0, 4] <= 4
fail		| [7, 8, 1, 0, 4] <= 3
fail		| [7, 8, 1, 0, 4, 3] <= 5
hit		| [7, 8, 1, 0, 4, 3, 5] <= 4
fail		| [7, 8, 1, 0, 4, 3, 5] <= 6
fail		| [8, 1, 0, 4, 3, 5, 6] <= 2
hit		| [1, 0, 4, 3, 5, 6, 2] <= 3
hit		| [1, 0, 4, 3, 5, 6, 2] <= 4
hit		| [1, 0, 4, 3, 5, 6, 2] <= 5
hit		| [1, 0, 4, 3, 5, 6, 2] <= 6
hit		| [1, 0, 4, 3, 5, 6, 2] <= 1
hit		| [1, 0, 4, 3, 5, 6, 2] <= 2
hit		| [1, 0, 4, 3, 5, 6, 2] <= 4
----------------------------------------------------------------------------------------------------
hit  cache  =  [1, 7, 4, 4, 3, 4, 5, 6, 1, 2, 4]
hit  times  =  11
hit  rate   = 55.00%
fail cache  =  [7, 8, 1, 0, 4, 3, 5, 6, 2]
fail times  =  9


In [ ]:
print('LRU')
simulation(pages, 7, arg_fn = lru)

LRU
是否擊中	| 分頁編號 <= 輸入
hit/fail	| [1, 2, 3, 4, 5, 6, 7] <= page
----------------------------------------------------------------------------------------------------
fail		| [] <= 7
fail		| [7] <= 8
fail		| [7, 8] <= 1
fail		| [7, 8, 1] <= 0
hit		| [7, 8, 1, 0] <= 1
fail		| [7, 8, 1, 0] <= 4
hit		| [7, 8, 1, 0, 4] <= 7
hit		| [7, 8, 1, 0, 4] <= 4
fail		| [7, 8, 1, 0, 4] <= 3
fail		| [7, 8, 1, 0, 4, 3] <= 5
hit		| [7, 8, 1, 0, 4, 3, 5] <= 4
fail		| [7, 8, 1, 0, 4, 3, 5] <= 6
fail		| [7, 1, 0, 4, 3, 5, 6] <= 2
hit		| [7, 1, 4, 3, 5, 6, 2] <= 3
hit		| [7, 1, 4, 3, 5, 6, 2] <= 4
hit		| [7, 1, 4, 3, 5, 6, 2] <= 5
hit		| [7, 1, 4, 3, 5, 6, 2] <= 6
hit		| [7, 1, 4, 3, 5, 6, 2] <= 1
hit		| [7, 1, 4, 3, 5, 6, 2] <= 2
hit		| [7, 1, 4, 3, 5, 6, 2] <= 4
----------------------------------------------------------------------------------------------------
hit  cache  =  [1, 7, 4, 4, 3, 4, 5, 6, 1, 2, 4]
hit  times  =  11
hit  rate   = 55.00%
fail cache  =  [7, 8, 1, 0, 4, 3, 5, 6, 2]
fail times  =  9
f

In [ ]:
print('OPT')
simulation(pages, 7, arg_fn = opt)

OPT
是否擊中	| 分頁編號 <= 輸入
hit/fail	| [1, 2, 3, 4, 5, 6, 7] <= page
----------------------------------------------------------------------------------------------------
fail		| [] <= 7
fail		| [7] <= 8
fail		| [7, 8] <= 1
fail		| [7, 8, 1] <= 0
hit		| [7, 8, 1, 0] <= 1
fail		| [7, 8, 1, 0] <= 4
hit		| [7, 8, 1, 0, 4] <= 7
hit		| [7, 8, 1, 0, 4] <= 4
fail		| [7, 8, 1, 0, 4] <= 3
fail		| [7, 8, 1, 0, 4, 3] <= 5
hit		| [7, 8, 1, 0, 4, 3, 5] <= 4
fail		| [7, 8, 1, 0, 4, 3, 5] <= 6
fail		| [8, 1, 0, 4, 3, 5, 6] <= 2
hit		| [1, 0, 4, 3, 5, 6, 2] <= 3
hit		| [1, 0, 4, 3, 5, 6, 2] <= 4
hit		| [1, 0, 4, 3, 5, 6, 2] <= 5
hit		| [1, 0, 4, 3, 5, 6, 2] <= 6
hit		| [1, 0, 4, 3, 5, 6, 2] <= 1
hit		| [1, 0, 4, 3, 5, 6, 2] <= 2
hit		| [1, 0, 4, 3, 5, 6, 2] <= 4
----------------------------------------------------------------------------------------------------
hit  cache  =  [1, 7, 4, 4, 3, 4, 5, 6, 1, 2, 4]
hit  times  =  11
hit  rate   = 55.00%
fail cache  =  [7, 8, 1, 0, 4, 3, 5, 6, 2]
fail times  =  9
f